In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os
import glob
import seaborn as sns
sns.set_context("talk",font_scale=1.5)
sns.set_style('whitegrid')

# User config

In [ ]:
# Paths to user files
data_dir = os.path.normpath(r'F:\Work\e\Data\Obs\Canada_Project_Sites\CSAS_data') # Where to store data on local computer

# Create paths

In [ ]:
hourly_merged = os.path.join(data_dir,'merged','Hourly_Merged.nc')

In [ ]:
QC_dir = os.path.join(data_dir,'QC')
# Make if does not exist
if not os.path.exists(QC_dir):
    os.makedirs(QC_dir)
netcdf_file_out = os.path.join(QC_dir, 'Hourly_QC.nc')

# QC merged data

In [ ]:
ds_merged = xr.open_dataset(hourly_merged)

In [ ]:
ds_merged

In [ ]:
plt.figure()
plt.plot(ds_merged.Time_UTC,ds_merged.SnowWaterEquivelentA.values);
plt.figure()
plt.plot(ds_merged.Time_UTC,ds_merged.SnowDepthA.values);
plt.figure()
plt.plot(ds_merged.Time_UTC,ds_merged.AirtemperatureA.values);
plt.figure()
plt.plot(ds_merged.Time_UTC,ds_merged.CummulativePrecipitationA.values);

In [ ]:
ds = ds_merged.copy() # Make copy to QC (allows comparison at end)

# Quality control data here

In [ ]:
## Quality Control

## Max and min
# SnowWaterEquivelentA
max_swe = 3 # m
min_swe = 0 # m
ds['SnowWaterEquivelentA'] = ds.SnowWaterEquivelentA.where((ds.SnowWaterEquivelentA<max_swe) &  (ds.SnowWaterEquivelentA>min_swe))

# SD
max_sd = 5.50 # m
min_sd = 0 # m
ds['SnowDepthA'] = ds.SnowDepthA.where((ds.SnowDepthA<max_sd) &                                                              
                                               (ds.SnowDepthA>min_sd))

# Precip
max_p = 3 # m
min_p = 0 # m
ds['CummulativePrecipitationAA'] = ds.CummulativePrecipitationA.where((ds.CummulativePrecipitationA<max_p) &                                                              
                                               (ds.CummulativePrecipitationA>min_p))

# Tair
max_tar = 40 # C
min_tar = -50 # C
ds['AirtemperatureA'] = ds.AirtemperatureA.where((ds.AirtemperatureA<max_tar) & 
                                                        (ds.AirtemperatureA>min_tar))

In [ ]:
def remove_outliers_via_filter(x,threshold,window):
    if(sum(np.isnan(x.values))/len(x.values)>0.9): # Mostly nan, just return x, otherwise filter fails
        return x
    else: # Have some data, apply the median filter and remove differences greater than threshold
        # Apply median filter
        temp = x.to_series().rolling(window=window, center=True).median().fillna(method='bfill').fillna(method='ffill')
        # Take difference between filter and orig data
        difference = np.abs(x.to_series() - temp)
        # Find those data values that the diff was less than the user supplied threshold
        inlier_idx  = difference < threshold
        return x.where(inlier_idx)

In [ ]:
## ROC - Use median filter to find values

# SD
SD_ROC_threshold = 50/100 # m/hr
SD_ROC_window = 10 # dt (hrs)
ds['SnowDepthA'] = ds.SnowDepthA.groupby('staID').apply(lambda x: remove_outliers_via_filter(x,SD_ROC_threshold,SD_ROC_window))

# SnowWaterEquivelentA
SnowWaterEquivelentA_ROC_threshold = 5/1000 # m/hr
SnowWaterEquivelentA_ROC_window = 10 # dt (hrs)
ds['SnowWaterEquivelentA'] = ds.SnowWaterEquivelentA.groupby('staID').apply(lambda x: remove_outliers_via_filter(x,SnowWaterEquivelentA_ROC_threshold,SnowWaterEquivelentA_ROC_window))

# Accumulated Precip (total)
P_ROC_threshold = 20/1000 # m/hr
P_ROC_window = 48 # dt (hrs)
ds['CummulativePrecipitationA'] = ds.CummulativePrecipitationA.groupby('staID').apply(lambda x: remove_outliers_via_filter(x,P_ROC_threshold,P_ROC_window))

# Air temperature
T_ROC_threshold = 10 # C/hr
T_ROC_window = 6 # dt (hrs)
ds['AirtemperatureA'] = ds.AirtemperatureA.groupby('staID').apply(lambda x: remove_outliers_via_filter(x,T_ROC_threshold,T_ROC_window))

In [ ]:
# import mpld3
# mpld3.enable_notebook()

In [ ]:
plt.plot(ds.Time_UTC,ds.SnowWaterEquivelentA.values);

In [ ]:
plt.plot(ds.Time_UTC,ds.SnowDepthA.values);

In [ ]:
plt.plot(ds.Time_UTC,ds.AirtemperatureA.values);

In [ ]:
plt.plot(ds.Time_UTC,ds.CummulativePrecipitationA.values);

In [ ]:
# Save as netcdf file
ds.to_netcdf(netcdf_file_out)
print(netcdf_file_out)